### 루프에 대한 개념 이해
#### RNN 정방향 계산
> 이 RNN은 크기가 (timesteps, input_features)인 2D 텐서로 인코딩된 벡터의 시퀀스를 입력 받는다. 이 시퀀스는 타임스템을 따라 반복된다. 각 타임스텝 t에서 현재상태와 ((input_features, 크기의) 입력을 연결하여 출력을 계산한다. 그다음 이 출력을 다음스텝의 상태로 설정한다. 첫 번째 타임스텝에서는 이전 출력이 정의되지 않으므로 현재 상태가 없다. 이때는 네트워크의 **초기 상태**인 0 벡터로 상태를 초기화한다.

### 의사코드로 표현한 RNN

In [ ]:
state_t = 0 # 타임스텝 t의 상태
for input_t in input_sequence: # 시퀀스의 원소 반복
    output_t = f(input_t, state_t)
    state_t = output_t # 출력은 다음 반복을 위한 상태가 된다.

> 여기서 f 함수는 입력과 상태를 출력으로 변환한다. 이를 2개의 행렬 W와 U 그리고 편향 벡터를 사용하는 변환으로 바꿀 수 있다.

### 좀 더 자세한 의사코드로 표현한 RNN

In [ ]:
state_t = 0
for input_t in input_sequence:
    output_t = activation(dot(W, input_t) + dot(U, state_t) + b)
    state_t = output_t

### 넘파이로 구현한 간단한 RNN의 정방형 계산

In [ ]:
import numpy as np

timesteps = 100 # 입력 시퀀스애 있는 타임스텝의 수
input_features = 32 # 입력 특성의 차원
output_features = 64 # 출력 특성의 차원

inputs = np.random.random((timesteps, input_features)) # 입력 데이터 : 예제를 위해 생성한 난수

state_t = np.zeros((output_features,)) # 모두 0인 초기상태의 벡터

# 랜덤한 가중치 행렬을 만든다.
W = np.random.random((output_features, input_features))
U = np.random.random((output_features, output_features))
b = np.random.random((output_features))

successive_outputs = []
for input_t in inputs:
    output_t = np.tanh(np.dot(W, input_t) + np.dot(U, state_t) + b)
    # 입력상태와 현재 상태를 연결하여 현재 출력을 얻는다.
    successive_outputs.append(output_t)
    # 이 출력을 리스트에 저장한다.
    state_t = output_t
    # 다음 타임 스텝을 위해 네트워크의 상태를 업데이트 한다.
    # 기본 RNN의 상태를 은닉 상태라고도 부른다. 은닉 상태는 이전 타임스텝의 출력이다.
    
final_ouput_sequence = np.stack(successive_outputs, axis=0)
# 최종 출력은 크기가 (timesteps, ouput_features)인 2D텐서이다.

> 요약하면 RNN은 반복할 때 이전에 계산한 정보를 재사용하는 for 루프 비슷한 것이다.
#### 이 예에서 최종출력은 (timesteps, output_features) 크기인 2D텐서인데, 각 타임스텝은 시간 t에서의 출력을 나타낸다. 출력 텐서의 각 타임스텝 t에는 입력 시퀀스에 있는 타임스텝에서 t까지 전체 과거에 대한 정보를 담고 있다. 이런 이유 때문에 많은 경우 전체 출력 시퀀스가 필요하지 않다.